# Этап 1: Продвинутый анализ потенциала сжатия

Мы уже знаем размер модели. Теперь давайте выясним, насколько она «хрупкая» и где прячутся главные враги квантования.

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from src.model import GPTLanguageModel, device

# Загружаем модель (желательно уже обученную)
model = GPTLanguageModel().to(device)
try:
    model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
    print("✅ Загружены обученные веса из model_ckpt.pt")
except:
    print("⚠️ Чекпоинт не найден, используем случайные веса (результаты будут менее точными)")

model.eval()

### 1. Анализ Выбросов (Outliers Analysis)
Выбросы — это значения весов, которые сильно выбиваются из общего распределения. Они заставляют квантователь выбирать слишком широкий диапазон, из-за чего 99% обычных весов теряют точность.

In [ ]:
def analyze_outliers(mdl, threshold_sigma=3.0):
    print(f"{'Layer Name':<40} | {'Max Val':<10} | {'Outliers %':<10}")
    print("-" * 70)
    
    for name, param in mdl.named_parameters():
        if 'weight' in name and param.dim() > 1:
            data = param.data.cpu().numpy().flatten()
            mean = np.mean(data)
            std = np.std(data)
            
            # Считаем сколько весов выходят за пределы N сигм
            outliers = np.sum(np.abs(data - mean) > threshold_sigma * std)
            outlier_pct = (outliers / len(data)) * 100
            
            print(f"{name[:40]:<40} | {np.max(np.abs(data)):<10.4f} | {outlier_pct:<10.2f}%")

analyze_outliers(model)

### 2. Saliency: Какие слои важнее?
Мы добавим к весам шум разной амплитуды и посмотрим, как это влияет на Loss. Это покажет нам «чувствительность» модели к порче данных.

In [ ]:
from src.model import get_batch

def measure_sensitivity(mdl, noise_level=0.01):
    xb, yb = get_batch('val')
    
    # Базовый лосс
    with torch.no_grad():
        _, base_loss = mdl(xb, yb)
    
    print(f"Базовый Loss на валидации: {base_loss.item():.4f}\n")
    print(f"{'Layer to noise':<40} | {'Delta Loss':<10} | {'Sensitivity'}")
    print("-" * 75)

    for name, param in mdl.named_parameters():
        if 'weight' in name and param.dim() > 1:
            orig_data = param.data.clone()
            
            # Добавляем шум
            noise = torch.randn_like(param.data) * noise_level
            param.data.add_(noise)
            
            with torch.no_grad():
                _, new_loss = mdl(xb, yb)
            
            delta = new_loss.item() - base_loss.item()
            
            # Оценка: чем выше Sensitivity, тем опаснее сжимать этот слой
            sensitivity = "🔴 HIGH" if delta > 0.05 else ("🟡 MED" if delta > 0.01 else "🟢 LOW")
            
            print(f"{name[:40]:<40} | {delta:<10.4f} | {sensitivity}")
            
            # Возвращаем веса назад!
            param.data.copy_(orig_data)

measure_sensitivity(model)

### Выводы Этапа 1:
1. **Outliers**: Если процент выбросов высок, обычное квантование (Min-Max) сильно уронит точность. Нам понадобятся методы вроде **Clamping** или **SmoothQuant**.
2. **Sensitivity**: Слои с пометкой 🔴 HIGH нужно сохранять в максимальной точности (например, оставить в FP16) или использовать для них большее количество бит.